# Cohesive energy
We will use the `KIMPotential` module to directly predict the energy of the system with respect to the lattice constant.
We will then demonstrate an easier, more principled way of doing the same using the `KIMRun` module.

```{note}
As per the OpenKIM defination, Cohesive energy is the negative per-atom potential energy of the material
```

## Computing the Energy vs Lattice constant manually

In [ ]:
%matplotlib inline

from orchestrator.potential.kim import KIMPotential
from orchestrator.target_property.kimrun import KIMRun
import numpy as np
from ase.build import bulk
import matplotlib.pyplot as plt

You can list all of the installed models using the `list_saved_potentials`,

In [ ]:
KIMPotential.list_saved_potentials()

This should print all the available drivers and potentials on your system. Next step is to initialize the correct potential using the KIMPotential module. `KIMPotential` needs the model name, list of species, model driver name, and the path to `kim-api-collections-management` utility. This utility should be available in your environment path, so simply listing it should be sufficient. 

In [ ]:
potential = KIMPotential(
    "SW_StillingerWeber_1985_Si__MO_405512056662_006",
    ["Si"],
    "SW__MD_335816936951_005",
    "kim-api-collections-management",
)

Post initialization of the potential, you can use its `evaluate` method to directly compute the energy from `ase.Atoms` objects.
Steps are:
1. Create a grid of latice constants
2. Create list of bulk configs of `ase.Atoms`
3. Compute energy of each config 

In [ ]:
# Set up the lattice constant grid (A)
a0    = 5.43           # approximate equilibrium lattice constant for Si
da    = 0.15           # half-width of scan range
npts  = 31             # number of points in the scan
a_grid = np.linspace(a0 - da, a0 + da, npts)
si_structures = [ bulk('Si', 'diamond', a=a) for a in a_grid ]
# evaluate energies and normalize by number of atoms
energies = [-potential.evaluate(c)[0]/len(c) for c in si_structures]

Now plotting it should show clearly show you the "Energy vs Lattice constant" curve, with minima at about 5.44 A.

In [ ]:
plt.plot(a_grid, np.array(energies), 'o-')
xlim, ylim = plt.xlim(), plt.ylim()
plt.xlabel('Lattice constant (A)')
plt.ylabel('Energy (eV)')
plt.title('Lattice constant scan for Si using KIM potential')
plt.savefig('lattice_constant_scan.png')

## KIMRun: More dogmatic method for calculating properties

Most of the common tests/property calculations, are already implemented in OpenKIM tests. `KIMRun` is a utility providing easy access to all of these tests. `KIMRun.calculate_property` can compute results for any KIM potential. It is also possible to pass the name of a potential saved in kimkit as the potential argument for `KIMRun.calculate_property`.

In [ ]:
my_kimrun = KIMRun()

potential = KIMPotential(
    "SW_StillingerWeber_1985_Si__MO_405512056662_006",
    ["Si"],
    "SW__MD_335816936951_005",
    "kim-api-collections-management",
)

results = my_kimrun.calculate_property(
        potential = potential,
        get_test_result_args = [
                {
                    "test": ["CohesiveEnergyVsLatticeConstant_diamond_Si__TE_973027833948_004"],
                    "prop": ["cohesive-energy-relation-cubic-crystal"],
                    "keys": ["cohesive-potential-energy", "a"],
                    "units": ["eV", "angstrom"]
                },
             ]
        )


Here, `get_test_result_args` expects the following
| key | explanation | example |
|:---:|:-----------:|:-------:|
|`test` | Openkim test to run (https://openkim.org/browse/tests/by-species) |CohesiveEnergyVsLatticeConstant_diamond_Si__TE_973027833948_004|
|`prop` | property you want to compute, properties can be accessed by clicking on the desired test on link above, then clicking on hyperlikned `Properties` on the Test page | `cohesive-energy-relation-cubic-crystal`  |
|`keys` | Property defination keys to report; `cohesive-energy-relation-cubic-crystal`  reports 4 keys you can pick from, `a`, `basis-atom-coordinates`, `cohesive-potential-energy`, `species`. From these we need `a`, and `cohesive-potential-energy`. | `cohesive-potential-energy`, `a` |
|`units`| units of the reported keys properties |`eV`, `angstrom`|

The output will be returned in a dictionary, with the values accessed via the
key 'property_value'. The results are nested lists, with the first index
associated with the number of tests (in this case just one), the second index
associated with the results for a given test (again just one in this case), 
and the following index corresponding to the requested keys (energy and lattice
parameter in this example)

In [ ]:
# first nesting is getting results from the first (and only) test
# second nesting is getting results from the first (and only) iteration of the test
# thrid nesting is lists of the requested keys
energy = results["property_value"][0][0][0]
a = results["property_value"][0][0][1]

energy = np.squeeze(energy)
a = np.squeeze(a)


fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(a, energy, 'o-')
ax1.set_xlim(xlim)
ax1.set_ylim(ylim)
ax2.plot(a, energy, 'o-')
fig.supxlabel('Lattice constant (A)')
fig.supylabel('Energy (eV)')
fig.suptitle('Lattice constant scan for Si using KIM potential')
plt.tight_layout()
plt.savefig('cohesive_energy_lattice_const.png')
